## Experiment on Taxi Trip Data

### Load Libraries

In [1]:
import findspark
findspark.init()
import time
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql import DataFrame
import pyspark.sql.functions as F

### Configurations

In [2]:
def conf1_setup():
    conf = SparkConf() \
        .set('spark.driver.memory', '20G') \
        .set('spark.sql.autoBroadcastJoinThreshold', '-1') \
        .set('spark.sql.shuffle.partitions', '16') \
        .set('spark.sql.adaptive.enabled', 'false')

    spark_session = SparkSession\
        .builder\
        .master('local[8]')\
        .config(conf = conf)\
        .appName("conf1") \
        .getOrCreate()
    
    url = spark.sparkContext.uiWebUrl
    print("Spark web UI: ", url)
    
    return spark_session

In [3]:
def conf2_setup():
    conf = SparkConf() \
        .set('spark.driver.memory', '20G') \
        .set('spark.sql.shuffle.partitions', '16') \
        .set('spark.sql.adaptive.enabled', 'false') \
        
    spark_session = SparkSession\
        .builder\
        .master('local[8]')\
        .config(conf = conf)\
        .appName("conf2") \
        .getOrCreate()
    
    url = spark.sparkContext.uiWebUrl
    print("Spark web UI: ", url)
    
    return spark_session

In [4]:
def conf3_setup():
    conf = SparkConf() \
        .set('spark.driver.memory', '20G') \
        .set('spark.sql.shuffle.partitions', '16') \
        .set('spark.sql.autoBroadcastJoinThreshold', '-1') \
        .set('spark.sql.adaptive.enabled', 'true') \
        .set("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .set("spark.sql.adaptive.skewJoin.enabled", "true") \
        
    spark_session = SparkSession\
        .builder\
        .master('local[8]')\
        .config(conf = conf)\
        .appName("conf3") \
        .getOrCreate()
    
    url = spark.sparkContext.uiWebUrl
    print("Spark web UI: ", url)
    
    return spark_session

In [5]:
def conf4_setup():
    conf = SparkConf() \
        .set('spark.driver.memory', '20G') \
        .set('spark.sql.shuffle.partitions', '16') \
        .set('spark.sql.autoBroadcastJoinThreshold', '-1') \
        .set('spark.sql.adaptive.enabled', 'true') \
        .set("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .set("spark.sql.adaptive.skewJoin.enabled", "true") \
        .set("spark.sql.adaptive.skewJoin.skewedPartitionFactor", "3") \
        .set("spark.sql.adaptive.skewJoin.skewedPartitionThresholdInBytes", "256K")\
        .set("spark.sql.adaptive.coalescePartitions.minPartitionSize", "128K")
        
    spark_session = SparkSession\
        .builder\
        .master('local[8]')\
        .config(conf = conf)\
        .appName("conf4") \
        .getOrCreate()
    
    url = spark.sparkContext.uiWebUrl
    print("Spark web UI: ", url)
    
    return spark_session

### Queries

In [6]:
def conf1_query1():
    # set up spark session
    spark = conf1_setup()

    # read data
    taxi_trips = spark.read.parquet("file:///Users/jiashu/Documents/UW_CSE544/project/experiments_finals/data/data1/trips/*.parquet")
    location_lookup = spark.read.option('header', True).csv('file:///Users/jiashu/Documents/UW_CSE544/project/experiments_finals/data/data1/taxi_zone_lookup.csv')
    
    # manipulate data to make it more skew
    pu_loc_to_change = [236, 132, 161, 186, 142, 141, 48, 239, 170, 162, 230, 163, 79, 234, 263, 140, 238, 107, 68, 138, 229, 249,
    237, 164, 90, 43, 100, 246, 231, 262, 113, 233, 143, 137, 114, 264, 148, 151]
    taxi_trips = taxi_trips.withColumn('PULocationID',
                   F.when(F.col('PULocationID').isin(pu_loc_to_change), F.lit(237))
                   .otherwise(F.col('PULocationID')))
    
    """
    # check data skewness 
    taxi_trips\
    .groupBy('PULocationID')\
    .agg(F.count(F.lit(1)).alias("num_rows")).sort(F.col("num_rows").desc())\
    .show(truncate=False, n = 30)
    """
    
    # join two tables and group by
    taxi_trips.join(location_lookup, F.col('PULocationID') == F.col('LocationID'), 'inner')\
        .groupBy("Zone") \
        .agg(F.avg('trip_distance').alias("avg_trip_distance")) \
        .show(truncate = False, n = 500)

In [7]:
def conf1_query2():
    # set up spark session
    spark = conf1_setup()

    # read data
    taxi_trips = spark.read.parquet("file:///Users/jiashu/Documents/UW_CSE544/project/experiments_finals/data/data1/trips/*.parquet")
    location_lookup = spark.read.option('header', True).csv('file:///Users/jiashu/Documents/UW_CSE544/project/experiments_finals/data/data1/taxi_zone_lookup.csv')
     
    # manipulate data to make it more skew
    pu_loc_to_change = [236, 132, 161, 186, 142, 141, 48, 239, 170, 162, 230, 163, 79, 234, 263, 140, 238, 107, 68, 138, 229, 249,
    237, 164, 90, 43, 100, 246, 231, 262, 113, 233, 143, 137, 114, 264, 148, 151]
    taxi_trips = taxi_trips.withColumn('PULocationID',
                   F.when(F.col('PULocationID').isin(pu_loc_to_change), F.lit(237))
                   .otherwise(F.col('PULocationID')))
    
    # key salting
    salting_n = 20
    
    taxi_trips = taxi_trips.withColumn("row_num", F.dayofyear(F.col("tpep_pickup_datetime"))) \
                .withColumn("row_num", F.col('row_num')%salting_n)
        
    location_lookup = location_lookup \
        .withColumn('key2', F.array([F.lit(num) for num in range(0, salting_n)]))\
        .withColumn('key2', F.explode(F.col('key2')))
    
    # join two tables and group by
    taxi_trips.join(location_lookup, 
               (F.col('PULocationID') == F.col('LocationID')) & (F.col('row_num') == F.col('key2')),
               'inner')\
        .groupBy("Zone") \
        .agg(F.avg('trip_distance').alias("avg_trip_distance")) \
        .show(truncate = False, n = 500)

In [8]:
def conf1_query3():
    # set up spark session
    spark = conf1_setup()

    # read data
    taxi_trips = spark.read.parquet("file:///Users/jiashu/Documents/UW_CSE544/project/experiments_finals/data/data1/trips/*.parquet")
    location_lookup = spark.read.option('header', True).csv('file:///Users/jiashu/Documents/UW_CSE544/project/experiments_finals/data/data1/taxi_zone_lookup.csv')
     
    # manipulate data to make it more skew
    pu_loc_to_change = [236, 132, 161, 186, 142, 141, 48, 239, 170, 162, 230, 163, 79, 234, 263, 140, 238, 107, 68, 138, 229, 249,
    237, 164, 90, 43, 100, 246, 231, 262, 113, 233, 143, 137, 114, 264, 148, 151]
    taxi_trips = taxi_trips.withColumn('PULocationID',
                   F.when(F.col('PULocationID').isin(pu_loc_to_change), F.lit(237))
                   .otherwise(F.col('PULocationID')))
    
    # key salting
    salting_n = 40
    
    taxi_trips = taxi_trips.withColumn("row_num", F.dayofyear(F.col("tpep_pickup_datetime"))) \
                .withColumn("row_num", F.col('row_num')%salting_n)
        
    location_lookup = location_lookup \
        .withColumn('key2', F.array([F.lit(num) for num in range(0, salting_n)]))\
        .withColumn('key2', F.explode(F.col('key2')))
    
    # join two tables and group by
    taxi_trips.join(location_lookup, 
               (F.col('PULocationID') == F.col('LocationID')) & (F.col('row_num') == F.col('key2')),
               'inner')\
        .groupBy("Zone") \
        .agg(F.avg('trip_distance').alias("avg_trip_distance")) \
        .show(truncate = False, n = 500)

In [9]:
def conf2_query4():
    # set up spark session
    spark = conf2_setup()

    # read data
    taxi_trips = spark.read.parquet("file:///Users/jiashu/Documents/UW_CSE544/project/spark_trials/data/data1/trips/*.parquet")
    location_lookup = spark.read.option('header', True).csv('file:///Users/jiashu/Documents/UW_CSE544/project/spark_trials/data/data1/taxi_zone_lookup.csv')
    
    # manipulate data to make it more skew
    pu_loc_to_change = [236, 132, 161, 186, 142, 141, 48, 239, 170, 162, 230, 163, 79, 234, 263, 140, 238, 107, 68, 138, 229, 249,
    237, 164, 90, 43, 100, 246, 231, 262, 113, 233, 143, 137, 114, 264, 148, 151]
    taxi_trips = taxi_trips.withColumn('PULocationID',
                   F.when(F.col('PULocationID').isin(pu_loc_to_change), F.lit(237))
                   .otherwise(F.col('PULocationID')))
    
    # join two tables and group by
    taxi_trips.join(location_lookup, F.col('PULocationID') == F.col('LocationID'), 'inner')\
        .groupBy("Zone") \
        .agg(F.avg('trip_distance').alias("avg_trip_distance")) \
        .show(truncate = False, n = 500)

In [10]:
def conf3_query5():
    # set up spark session
    spark = conf3_setup()

    # read data
    taxi_trips = spark.read.parquet("file:///Users/jiashu/Documents/UW_CSE544/project/experiments_finals/data/data1/trips/*.parquet")
    location_lookup = spark.read.option('header', True).csv('file:///Users/jiashu/Documents/UW_CSE544/project/experiments_finals/data/data1/taxi_zone_lookup.csv')
     
    # manipulate data to make it more skew
    pu_loc_to_change = [236, 132, 161, 186, 142, 141, 48, 239, 170, 162, 230, 163, 79, 234, 263, 140, 238, 107, 68, 138, 229, 249,
    237, 164, 90, 43, 100, 246, 231, 262, 113, 233, 143, 137, 114, 264, 148, 151]
    taxi_trips = taxi_trips.withColumn('PULocationID',
                   F.when(F.col('PULocationID').isin(pu_loc_to_change), F.lit(237))
                   .otherwise(F.col('PULocationID')))
    
    # join two tables and group by
    taxi_trips.join(location_lookup, F.col('PULocationID') == F.col('LocationID'), 'inner')\
        .groupBy("Zone") \
        .agg(F.avg('trip_distance').alias("avg_trip_distance")) \
        .show(truncate = False, n = 500)

In [11]:
def conf3_query6():
    # set up spark session
    spark = conf3_setup()

    # read data
    taxi_trips = spark.read.parquet("file:///Users/jiashu/Documents/UW_CSE544/project/spark_trials/data/data1/trips/*.parquet")
    location_lookup = spark.read.option('header', True).csv('file:///Users/jiashu/Documents/UW_CSE544/project/spark_trials/data/data1/taxi_zone_lookup.csv')
    
    # manipulate data to make it more skew
    pu_loc_to_change = [236, 132, 161, 186, 142, 141, 48, 239, 170, 162, 230, 163, 79, 234, 263, 140, 238, 107, 68, 138, 229, 249,
    237, 164, 90, 43, 100, 246, 231, 262, 113, 233, 143, 137, 114, 264, 148, 151]
    taxi_trips = taxi_trips.withColumn('PULocationID',
                   F.when(F.col('PULocationID').isin(pu_loc_to_change), F.lit(237))
                   .otherwise(F.col('PULocationID')))
    
    # key salting
    salting_n = 40
    
    taxi_trips = taxi_trips.withColumn("row_num", F.dayofyear(F.col("tpep_pickup_datetime"))) \
                .withColumn("row_num", F.col('row_num')%salting_n)
        
    location_lookup = location_lookup \
        .withColumn('key2', F.array([F.lit(num) for num in range(0, salting_n)]))\
        .withColumn('key2', F.explode(F.col('key2')))
    
    # join two tables and group by
    taxi_trips.join(location_lookup, 
               (F.col('PULocationID') == F.col('LocationID')) & (F.col('row_num') == F.col('key2')),
               'inner')\
        .groupBy("Zone") \
        .agg(F.avg('trip_distance').alias("avg_trip_distance")) \
        .show(truncate = False, n = 500)

In [12]:
def conf4_query7():
    # set up spark session
    spark = conf4_setup()

    # read data
    taxi_trips = spark.read.parquet("file:///Users/jiashu/Documents/UW_CSE544/project/experiments_finals/data/data1/trips/*.parquet")
    location_lookup = spark.read.option('header', True).csv('file:///Users/jiashu/Documents/UW_CSE544/project/experiments_finals/data/data1/taxi_zone_lookup.csv')
     
    # manipulate data to make it more skew
    pu_loc_to_change = [236, 132, 161, 186, 142, 141, 48, 239, 170, 162, 230, 163, 79, 234, 263, 140, 238, 107, 68, 138, 229, 249,
    237, 164, 90, 43, 100, 246, 231, 262, 113, 233, 143, 137, 114, 264, 148, 151]
    taxi_trips = taxi_trips.withColumn('PULocationID',
                   F.when(F.col('PULocationID').isin(pu_loc_to_change), F.lit(237))
                   .otherwise(F.col('PULocationID')))
    
    # join two tables and group by
    taxi_trips.join(location_lookup, F.col('PULocationID') == F.col('LocationID'), 'inner')\
        .groupBy("Zone") \
        .agg(F.avg('trip_distance').alias("avg_trip_distance")) \
        .show(truncate = False, n = 500)

In [13]:
def conf4_query8():
    # set up spark session
    spark = conf4_setup()

    # read data
    taxi_trips = spark.read.parquet("file:///Users/jiashu/Documents/UW_CSE544/project/spark_trials/data/data1/trips/*.parquet")
    location_lookup = spark.read.option('header', True).csv('file:///Users/jiashu/Documents/UW_CSE544/project/spark_trials/data/data1/taxi_zone_lookup.csv')
    
    # manipulate data to make it more skew
    pu_loc_to_change = [236, 132, 161, 186, 142, 141, 48, 239, 170, 162, 230, 163, 79, 234, 263, 140, 238, 107, 68, 138, 229, 249,
    237, 164, 90, 43, 100, 246, 231, 262, 113, 233, 143, 137, 114, 264, 148, 151]
    taxi_trips = taxi_trips.withColumn('PULocationID',
                   F.when(F.col('PULocationID').isin(pu_loc_to_change), F.lit(237))
                   .otherwise(F.col('PULocationID')))
    
    # key salting
    salting_n = 40
    
    taxi_trips = taxi_trips.withColumn("row_num", F.dayofyear(F.col("tpep_pickup_datetime"))) \
                .withColumn("row_num", F.col('row_num')%salting_n)
        
    location_lookup = location_lookup \
        .withColumn('key2', F.array([F.lit(num) for num in range(0, salting_n)]))\
        .withColumn('key2', F.explode(F.col('key2')))
    
    # join two tables and group by
    taxi_trips.join(location_lookup, 
               (F.col('PULocationID') == F.col('LocationID')) & (F.col('row_num') == F.col('key2')),
               'inner')\
        .groupBy("Zone") \
        .agg(F.avg('trip_distance').alias("avg_trip_distance")) \
        .show(truncate = False, n = 500)

In [14]:
if __name__ == '__main__':
    start_time = time.time()
    
    conf1_query1()
    # conf1_query2()
    # conf1_query3()
    
    # conf2_query4()
    
    # conf3_query5()
    # conf3_query6()
    
    # conf4_query7()
    # conf4_query8()
    
    end_time = time.time()
    print(f"Elapsed_time: {end_time - start_time} seconds")

24/03/06 11:16:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Spark web UI:  http://10.19.49.31:4040


24/03/06 11:16:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/06 11:16:33 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/06 11:16:33 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/06 11:16:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/03/06 11:16:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+---------------------------------------------+------------------+
|Zone                                         |avg_trip_distance |
+---------------------------------------------+------------------+
|Queens Village                               |6.2417340286831795|
|Williamsburg (South Side)                    |4.283125504206519 |
|Whitestone                                   |6.940480349344978 |
|Hudson Sq                                    |10.240936320161019|
|Canarsie                                     |7.43429101707498  |
|Hunts Point                                  |4.27814465408805  |
|Williamsburg (North Side)                    |18.768557396567047|
|Jamaica Bay                                  |17.452352941176468|
|East Flushing                                |3.6545312500000002|
|Freshkills Park                              |8.996666666666666 |
|Spuyten Duyvil/Kingsbridge                   |4.836102635228847 |
|Arden Heights                                |11.306793103448

24/03/06 11:30:41 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 566175 ms exceeds timeout 120000 ms
24/03/06 11:30:41 WARN SparkContext: Killing executors is not supported by current scheduler.
24/03/06 11:30:45 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

...

24/03/06 11:55:20 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:295)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:1928)

24/03/06 11:55:30 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

24/03/06 11:55:40 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:295)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:1928)

24/03/06 11:55:50 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:295)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:1928)

24/03/06 11:57:46 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE